<a href="https://colab.research.google.com/github/mordor-ai/M2-MIASHS-DIVERS/blob/main/TP2_M2_2020_2021etudiant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
class_names = ['Avion', 'auto', 'oiseau', 'chat', 'biche',
               'chien', 'grenouille', 'cheval', 'bateau', 'camion']
               
train_images, test_images = train_images / 255.0, test_images / 255.0

170500096/170498071 [==============================] - 3s 0us/step


Pour comprendre comment ajouter des régularisations à vos réseaux allez voir les pages : 

https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/L1

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D

Implémenter le réseau suivant (la fonction d'activation est une ReLU) : 


```

Layer (type)                 Output Shape              Param #   

conv2d_44 (Conv2D)           (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 13, 13, 128)       73856     
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 11, 11, 128)       147584    
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 5, 5, 128)         0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 3, 3, 128)         147584    
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 1, 1, 128)         147584    
_________________________________________________________________
flatten_10 (Flatten)         (None, 128)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_21 (Dense)             (None, 10)                650       
```

Entraînez le en utilisant les paramètres par défaut de l'optimiseur Adam pendant 20 époques. 

**Notez les performances et les valeurs finales de la loss de train et de la loss de test**

Modifiez votre réseau pour effectuer les tests suivants : 


*   Une régularisation L1 avec un poids de 0.00001 sur chaque couche
*   Une régularisation L2 avec un poids de 0.001 sur chaque couche
*   Une régularisation L2 avec un poids de 0.001 sur chaque couche sauf les deux premières
*   Une régularisation L2 avec un poids de 0.01 sur chaque couche

**Que pouvez vous conclure des résultats obtenus ?**







In [ ]:
def createModel(gammaL1, gammaL2, first = False):
  randomName = "conv"+str(random.random())
  model = models.Sequential()

  model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3), 
                          bias_regularizer=tf.keras.regularizers.L1L2(l2=gammaL2, l1=gammaL1) if first else None, #64
                          kernel_regularizer=tf.keras.regularizers.L1L2(l2=gammaL2, l1=gammaL1) if first else None))#3x64x3x3
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu', 
                          bias_regularizer=tf.keras.regularizers.L1L2(l2=gammaL2, l1=gammaL1) if first else None, 
                          kernel_regularizer=tf.keras.regularizers.L1L2(l2=gammaL2, l1=gammaL1) if first else None))
  model.add(layers.Conv2D(128, (3, 3), activation='relu', 
                          kernel_regularizer=tf.keras.regularizers.L1L2(l2=gammaL2, l1=gammaL1), 
                          bias_regularizer=tf.keras.regularizers.L1L2(l2=gammaL2, l1=gammaL1)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu', 
                          kernel_regularizer=tf.keras.regularizers.L1L2(l2=gammaL2, l1=gammaL1), 
                          bias_regularizer=tf.keras.regularizers.L1L2(l2=gammaL2, l1=gammaL1)))

  model.add(layers.Conv2D(128, (3, 3), activation='relu', 
                            activity_regularizer=tf.keras.regularizers.L1L2(l2=gammaL2, l1=gammaL1), name=randomName, 
                            bias_regularizer=tf.keras.regularizers.L1L2(l2=gammaL2, l1=gammaL1)))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(10))

  model_tmp = tf.keras.Model(inputs=model.input, outputs=model.get_layer(randomName).output)

  #ICI 
  return model,model_tmp


def draw_projection(model_tmp, with_test, with_train, maximum=1000):
  data  = []
  labels = []
  marker = []
  if with_test:
    for i in range(0, test_images.shape[0], 100):
      data += list(model_tmp.predict(test_images[i:i+100]).reshape(100, -1))
      labels += list(test_labels[i:i+100])
    marker +=[1] * test_labels.shape[0]


  if with_train:
    for i in range(0, train_images.shape[0], 100):
      data += list(model_tmp.predict(train_images[i:i+100]).reshape(100, -1))
      labels += list(train_labels[i:i+100])
    marker +=[0] * train_labels.shape[0]

  data = np.array(data)
  labels=np.array(labels)
  marker=np.array(marker)

  index = np.arange(0, int(labels.shape[0]))
  np.random.shuffle(index)
  
  data   = data[index[:maximum]]
  labels = labels[index[:maximum]]
  marker = marker[index[:maximum]]


  data_tsne = TSNE(n_components=2).fit_transform(data)
  data_pca = PCA(n_components=2).fit_transform(data)
  if with_train:
    plt.subplot(1, 2, 1)
    plt.scatter(data_pca[marker==0, 0], data_pca[marker==0, 1], c=labels[marker==0], marker="x")
    plt.subplot(1, 2, 2)
    plt.scatter(data_tsne[marker==0, 0], data_tsne[marker==0, 1], c=labels[marker==0], marker="x")
    plt.colorbar()
  if with_test:
    plt.subplot(1, 2, 1)
    plt.scatter(data_pca[marker==1, 0], data_pca[marker==1, 1], c=labels[marker==1], marker="o")
    plt.subplot(1, 2, 2)
    plt.scatter(data_tsne[marker==1, 0], data_tsne[marker==1, 1], c=labels[marker==1], marker="o")
    plt.colorbar()


  fig = plt.gcf()
  fig.set_size_inches(20, 10)
  plt.show()

In [ ]:


params=[[0,0,False],[0.00001,0,True],[0,0.001,True],[0,0.001,False],[0,0.01,True]]
for param in params:
  model, model_tmp = createModel(param[0], param[1], param[2]) # on enleve le '_'
  print("L1 : ", param[0],"  L2 : ", param[1]," First : ", param[2])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])


  model.fit(train_images, train_labels, epochs=19, 
                      validation_data=(test_images, test_labels), verbose=0)
  model.fit(train_images, train_labels, epochs=1, 
                      validation_data=(test_images, test_labels), verbose=1)

  print ("TEST")
  draw_projection(model_tmp, with_test=True, with_train=False, maximum=7500)
  print ("TRAIN")
  draw_projection(model_tmp, with_test=False, with_train=True, maximum=7500)
  print ("TRAIN+TEST")
  draw_projection(model_tmp, with_test=True, with_train=True, maximum=10000)

  print ("-"*35)
  

L1 :  0   L2 :  0  First :  False
1563/1563 [==============================] - 8s 5ms/step - loss: 0.1731 - accuracy: 0.9409 - val_loss: 1.4125 - val_accuracy: 0.7217
-----------------------------------
L1 :  1e-05   L2 :  0  First :  True


KeyboardInterrupt: ignored

# Projection 2D

Pour chaque réseau précédemment entrainé afficher la représentation t-SNE et la PCA des données en considérant d'abord seulement la base d'entraînement puis seulement la base de test et enfin les bases d'entraînnement et de test ensembles. Vous utiliserez les cartes de caractéristiques de la 5ème convolution. 




# Pour allez plus loin
Ajoutez une régularisation, avec un poids que vous estimerez par l'expérimentation, sur la sortie de la 5ème convolution (et non sur les valeurs des poids). Après avoir effectué un entraînement, affichez la projection 2D de la 5ème convolution.



Répétez l'opération en choisissant une régularisation de type KL divergence avec un paramètre de sparcité égale à 0.001. **Vous devez implémenter cette régularisation et modifier le réseau pour que celle-ci fonctionne.**



---



---



---

# La classification, le deep learning et la réalité

Pour tous les TP qui suivent nous allons nous intéresser à un vrai problème de classification. Nous allons chercher à détecter les visages et à discriminer les personnes ayant un masque des autres. 

Pour cela nous allons utiliser trois bases trouvées sur internet. Chaque base comporte son format d'annotation et ses règles de labellisation. De plus les images étant issues du monde réel elles seront de différentes tailles et résolutions. 

1.   Téléchargez les données en utilisant le lien suivant : https://we.tl/t-UfSJCPGKZ3
2.   Complétez les classes 'Objet' et 'DataGenerator' afin que la classe 'DataGenerator' retourne des images avec la vérité terrain associée (en terme de segmentation pixel)




In [ ]:
!wget --user-agent Mozilla/4.0 'https://download.wetransfer.com//eu2/5a5d8f7599ac518caa5b1652e4c658a920201111135405/e0ad236d54b2ffb0c92751a9e46f0dcc6b6faff1/out_toto.tar.gz?cf=y&token=eyJhbGciOiJIUzI1NiJ9.eyJleHAiOjE2MDUxMzUwMjcsInVuaXF1ZSI6IjVhNWQ4Zjc1OTlhYzUxOGNhYTViMTY1MmU0YzY1OGE5MjAyMDExMTExMzU0MDUiLCJmaWxlbmFtZSI6Im91dF90b3RvLnRhci5neiIsIndheWJpbGxfdXJsIjoiaHR0cDovL3Byb2R1Y3Rpb24uYmFja2VuZC5zZXJ2aWNlLmV1LXdlc3QtMS53dDo5MjkyL3dheWJpbGwvdjEvc2Fya2FyL2ViOGI2NGEzNTQ1ODVmMDVkN2E5MGNlNzFiYmVkNzEyYzY5YmU0Zjc5MmJlNTgzZTVmNGM1MTkyZGRiZjBiYjc0ZmNkZThhZGQzMGY5MTY2ZDlkMTBmIiwiZmluZ2VycHJpbnQiOiJlMGFkMjM2ZDU0YjJmZmIwYzkyNzUxYTllNDZmMGRjYzZiNmZhZmYxIiwiY2FsbGJhY2siOiJ7XCJmb3JtZGF0YVwiOntcImFjdGlvblwiOlwiaHR0cDovL3Byb2R1Y3Rpb24uZnJvbnRlbmQuc2VydmljZS5ldS13ZXN0LTEud3Q6MzAwMC93ZWJob29rcy9iYWNrZW5kXCJ9LFwiZm9ybVwiOntcInRyYW5zZmVyX2lkXCI6XCI1YTVkOGY3NTk5YWM1MThjYWE1YjE2NTJlNGM2NThhOTIwMjAxMTExMTM1NDA1XCIsXCJkb3dubG9hZF9pZFwiOjEwNzEyMjE2NzI2fX0ifQ.qeU1IgXPulzP02kdsTE1DLsYtN5p45X3vPNxLMYv4dE' -O dest_file_name


--2020-11-11 22:40:52--  https://download.wetransfer.com//eu2/5a5d8f7599ac518caa5b1652e4c658a920201111135405/e0ad236d54b2ffb0c92751a9e46f0dcc6b6faff1/out_toto.tar.gz?cf=y&token=eyJhbGciOiJIUzI1NiJ9.eyJleHAiOjE2MDUxMzUwMjcsInVuaXF1ZSI6IjVhNWQ4Zjc1OTlhYzUxOGNhYTViMTY1MmU0YzY1OGE5MjAyMDExMTExMzU0MDUiLCJmaWxlbmFtZSI6Im91dF90b3RvLnRhci5neiIsIndheWJpbGxfdXJsIjoiaHR0cDovL3Byb2R1Y3Rpb24uYmFja2VuZC5zZXJ2aWNlLmV1LXdlc3QtMS53dDo5MjkyL3dheWJpbGwvdjEvc2Fya2FyL2ViOGI2NGEzNTQ1ODVmMDVkN2E5MGNlNzFiYmVkNzEyYzY5YmU0Zjc5MmJlNTgzZTVmNGM1MTkyZGRiZjBiYjc0ZmNkZThhZGQzMGY5MTY2ZDlkMTBmIiwiZmluZ2VycHJpbnQiOiJlMGFkMjM2ZDU0YjJmZmIwYzkyNzUxYTllNDZmMGRjYzZiNmZhZmYxIiwiY2FsbGJhY2siOiJ7XCJmb3JtZGF0YVwiOntcImFjdGlvblwiOlwiaHR0cDovL3Byb2R1Y3Rpb24uZnJvbnRlbmQuc2VydmljZS5ldS13ZXN0LTEud3Q6MzAwMC93ZWJob29rcy9iYWNrZW5kXCJ9LFwiZm9ybVwiOntcInRyYW5zZmVyX2lkXCI6XCI1YTVkOGY3NTk5YWM1MThjYWE1YjE2NTJlNGM2NThhOTIwMjAxMTExMTM1NDA1XCIsXCJkb3dubG9hZF9pZFwiOjEwNzEyMjE2NzI2fX0ifQ.qeU1IgXPulzP02kdsTE1DLsYtN5p45X3vPNxLMYv4dE
Resolving downl

In [ ]:
!wget --user-agent Mozilla/4.0 'https://download.wetransfer.com//eu2/9b751cb3099a84b606b98b9426215dcd20201217092809/e0ad236d54b2ffb0c92751a9e46f0dcc6b6faff1/out_toto.tar.gz?cf=y&token=eyJhbGciOiJIUzI1NiJ9.eyJleHAiOjE2MDgxOTgxNTcsInVuaXF1ZSI6IjliNzUxY2IzMDk5YTg0YjYwNmI5OGI5NDI2MjE1ZGNkMjAyMDEyMTcwOTI4MDkiLCJmaWxlbmFtZSI6Im91dF90b3RvLnRhci5neiIsIndheWJpbGxfdXJsIjoiaHR0cDovL3Byb2R1Y3Rpb24uYmFja2VuZC5zZXJ2aWNlLmV1LXdlc3QtMS53dDo5MjkyL3dheWJpbGwvdjEvc2Fya2FyLzRlZDU4ZmI2NmJlY2Y4Yzg2ZjFkNDNiYzdmN2ZkZDRmMWM2MWE0ZjNhNzczMTk4MGY1ZmZjYWU2YTIwYTg0ZDdmNGE0Mzc4ODZmMjRmMWNjM2MzOWQ5IiwiZmluZ2VycHJpbnQiOiJlMGFkMjM2ZDU0YjJmZmIwYzkyNzUxYTllNDZmMGRjYzZiNmZhZmYxIiwiY2FsbGJhY2siOiJ7XCJmb3JtZGF0YVwiOntcImFjdGlvblwiOlwiaHR0cDovL3Byb2R1Y3Rpb24uZnJvbnRlbmQuc2VydmljZS5ldS13ZXN0LTEud3Q6MzAwMC93ZWJob29rcy9iYWNrZW5kXCJ9LFwiZm9ybVwiOntcInRyYW5zZmVyX2lkXCI6XCI5Yjc1MWNiMzA5OWE4NGI2MDZiOThiOTQyNjIxNWRjZDIwMjAxMjE3MDkyODA5XCIsXCJkb3dubG9hZF9pZFwiOjExMDE5MTkwMTczfX0ifQ.GJ-39bdctQN51UKzcZ0moUG-3c-ZEmxhsyAKDnS7ZnA'

The name is too long, 834 chars total.
Trying to shorten...
New name is out_toto.tar.gz?cf=y&token=eyJhbGciOiJIUzI1NiJ9.eyJleHAiOjE2MDgxOTgxNTcsInVuaXF1ZSI6IjliNzUxY2IzMDk5YTg0YjYwNmI5OGI5NDI2MjE1ZGNkMjAyMDEyMTcwOTI4MDkiLCJmaWxlbmFtZSI6Im91dF90b3RvLnRhci5neiIsIndheWJpbGxfdXJsIjoiaHR0cDovL3Byb2R1Y3Rpb24uYmFj.
--2020-12-17 09:32:51--  https://download.wetransfer.com//eu2/9b751cb3099a84b606b98b9426215dcd20201217092809/e0ad236d54b2ffb0c92751a9e46f0dcc6b6faff1/out_toto.tar.gz?cf=y&token=eyJhbGciOiJIUzI1NiJ9.eyJleHAiOjE2MDgxOTgxNTcsInVuaXF1ZSI6IjliNzUxY2IzMDk5YTg0YjYwNmI5OGI5NDI2MjE1ZGNkMjAyMDEyMTcwOTI4MDkiLCJmaWxlbmFtZSI6Im91dF90b3RvLnRhci5neiIsIndheWJpbGxfdXJsIjoiaHR0cDovL3Byb2R1Y3Rpb24uYmFja2VuZC5zZXJ2aWNlLmV1LXdlc3QtMS53dDo5MjkyL3dheWJpbGwvdjEvc2Fya2FyLzRlZDU4ZmI2NmJlY2Y4Yzg2ZjFkNDNiYzdmN2ZkZDRmMWM2MWE0ZjNhNzczMTk4MGY1ZmZjYWU2YTIwYTg0ZDdmNGE0Mzc4ODZmMjRmMWNjM2MzOWQ5IiwiZmluZ2VycHJpbnQiOiJlMGFkMjM2ZDU0YjJmZmIwYzkyNzUxYTllNDZmMGRjYzZiNmZhZmYxIiwiY2FsbGJhY2siOiJ7XCJmb3JtZGF0YVwiOntcImFjdGlv

In [ ]:
import xml.etree.ElementTree as ET

FOLDER_DATA="./"
NCROSS = 5
CROSS  = 0
batch_size = 32
TAILLE_MAX=512

In [ ]:
class Objet:
    def __init__(self, xml_part):
        self.name = str(xml_part.find('name').text)
        self.xmin = int(xml_part.find('bndbox').find('xmin').text)
        self.xmax = int(xml_part.find('bndbox').find('xmax').text)
        self.ymin = int(xml_part.find('bndbox').find('ymin').text)
        self.ymax = int(xml_part.find('bndbox').find('ymax').text)
    def __str__(self):
        return self.name+" "+str(self.xmin)+" "+str(self.xmax)+" "+str(self.ymin)+" "+str(self.ymax)+" "+str(self.label)


class ReadXML:
    def __init__(self, ch):
        tree = ET.parse(ch)
        self.root = tree.getroot()
        self.filename=  self.root.find('filename').text
        if "annotations" in ch:
            self.image  = imageio.imread(FOLDER_DATA+'/images/'+self.filename)[:,:,:3]/255.0
        elif "medical-masks-dataset" in ch:
            self.image  = imageio.imread(FOLDER_DATA+'/medical-masks-dataset/images/'+self.filename)[:,:,:3]/255.0

        # On souhaite avoir une largeur/hauteur de taille maximale TAILLE_MAX ET chaque dimension doit-être un multiple de 32
        # self.image : L'image en entrée
        # self.image_zone : la vérité terrain (W, H, Nb Labels)
        # CODE A FAIRE 


        self.image_zone = np.zeros((self.image.shape[0], self.image.shape[1], 3), dtype=np.int32)
        self.image_zone[:,:, 0]=1
        
        for i, ob in enumerate(self.root.findall('object')):
            tmp = Objet(ob)
            label=-1
            if "annotations" in ch:
                if tmp.name=="with_mask":            
                    label=2
                elif tmp.name=="mask_weared_incorrect":
                    label=2
                elif tmp.name=="without_mask":
                    label=1
                else:
                    print("BUG")
            else:
                if tmp.name=="mask":            
                    label=2
                elif tmp.name=="poor":
                    label=2
                elif tmp.name=="none":
                    label=1
                else:
                    print("BUG")


            # On complète le code en mettant à jour le tableau 'self.image_zone'
            # CODE A FAIRE 

          

                    

In [ ]:

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data_link, batch_size=32,  shuffle=True):
        self.batch_size = batch_size
        self.data={}
        #Compléter l'initialisation du générateur
        # CODE A FAIRE                               
        self.indices = list(self.data.keys())
        self.shuffle = shuffle
        self.on_epoch_end()

        
    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        keys = [self.indices[k] for k in index]
        return self.__get_data(keys)

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

            
    def __get_data(self, keys):        
        pass
        #Compléter la méthode qui doit retourner : l'image (Nb Batch, H, W, 3) ,  la vérité terrain (Nb Batch, H, W, 3)
        # CODE A FAIRE 
                


In [ ]:

all_xml = glob.glob(FOLDER_DATA+'/medical-masks-dataset/labels/*.xml')+glob.glob(+FOLDER_DATA"/annotations/*.xml")
random.seed(42)
random.shuffle(all_xml)


sp = np.array_split(all_xml, NCROSS)
test_xml_imggb  = sp[CROSS]
train_xml_imggb = np.concatenate([x for i,x in enumerate(sp) if i!=CROSS], axis=0) 
train_xml_imggb = train_xml_imggb
test_xml_imggb  = test_xml_imggb

print ("TAILLE TEST ",len(test_xml_imggb) )
print ("TAILLE TRAIN ",len(train_xml_imggb) )
train = DataGenerator(data_link=train_xml_imggb, batch_size=batch_size,  shuffle=True)
val   = DataGenerator(data_link=test_xml_imggb, batch_size=batch_size,  shuffle=True)
